# Benchmarking tflite models  

This notebook parses the benchmarking.txt files generated by: 

 [https://www.tensorflow.org/lite/performance/measurement](https://www.tensorflow.org/lite/performance/measurement)  

 TensorFlow Lite benchmark tools currently measure and calculate statistics for the following important performance metrics:

- Initialization time
- Inference time of warmup state
- Inference time of steady state
- Memory usage during initialization time
- Overall memory usage

In [1]:
import os, sys, math, datetime
import pathlib
from pathlib import Path

# import workbench.config.config
from workbench.config.config import initialize
from workbench.utils.utils import create_filepaths

In [2]:
import re
from matplotlib import pyplot as plt
#import plotly.express as px
import pandas as pd


# enable plotly in VS Studio Code
#import plotly.io as pio
#pio.renderers.default = "notebook_connected"
#pio.renderers.default = "plotly_mimetype+notebook"

import wandb

In [3]:
# Configure pandas to show all columns & rows
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [4]:
models_dir = initialize()

In [5]:
automated = False
id = id = wandb.sdk.lib.runid.generate_id() 

global model_name
#model_name = "efficientNetB0_1_96_c1_o3_keras"
#model_name = "mobilenetv2_0.1_96_c1_o3_l1"
model_name = "mobilenetv1_0.1_96_c3_o3_l2"
#model_name = "shufflenetv2tiny_0.1_96_c3_o3_f4l1024"

models_path, models_summary_path, models_image_path, models_layer_df_path, models_tf_path, models_tflite_path, models_tflite_opt_path = create_filepaths(model_name)

/mnt/i/tinyml/tiny_cnn/models


In [6]:
models_benchmark_path = models_dir.joinpath(model_name, f"{model_name}_benchmark.txt")
models_benchmark_path

PosixPath('/mnt/i/tinyml/tiny_cnn/models/mobilenetv1_0.1_96_c3_o3_l2/mobilenetv1_0.1_96_c3_o3_l2_benchmark.txt')

# Helper functions

In [7]:
def clean_model_summary(filepath): 

    if not filepath.is_file():
        print(f"This file does not exist: {filepath}")
        return None

    else:
        clean_lines = []
        # Parse the MLTK model summary to grab important metrics   
        with open(filepath, "r", encoding="latin-1") as f:
            lines = f.readlines() # list containing lines of file
            for line in lines:
                line = line.strip() # remove leading/trailing white spaces
                if len(line)> 0:
                    
                    clean_lines.append(line)
                else:
                    pass
            #columns = [] # To store column names
        return clean_lines

In [8]:
def clean_column_names(df):
    cols = df.columns

    clean_cols = []
    for col in cols:
        col = col.strip()
        col = col.replace("[" , "")
        col = col.replace("]" , "")   
        clean_cols.append(col)
        
    return clean_cols

In [9]:
def string_percent_to_float(x):
    x = str(x).strip("%")
    return float(x)

In [10]:
def remove_tabs(text):
    """removes tabs from a list of strings

    Args:
        text (list(str)): list of strings that contains tabs

    Returns:
        list(str): list of strings without tabs
    """
    split_text= []
    for l in text:
        split_text.append((l.split("\t")))
        
    return split_text

# Parsing the model file

if models_benchmark_path.is_file():
    

In [11]:
lines = clean_model_summary(models_benchmark_path)
lines

['STARTING!',
 'Log parameter values verbosely: [0]',
 'Num threads: [1]',
 'Graph: [/mnt/i/tinyml/tiny_cnn/models/mobilenetv1_0.1_96_c3_o3_l2/mobilenetv1_0.1_96_c3_o3_l2_INT8.tflite]',
 'Enable op profiling: [1]',
 '#threads used for CPU inference: [1]',
 'Loaded model /mnt/i/tinyml/tiny_cnn/models/mobilenetv1_0.1_96_c3_o3_l2/mobilenetv1_0.1_96_c3_o3_l2_INT8.tflite',
 'The input model file size (MB): 0.07144',
 'Initialized session in 154.788ms.',
 'Running benchmark for at least 1 iterations and at least 0.5 seconds but terminate if exceeding 150 seconds.',
 'count=735 first=2826 curr=589 min=554 max=2826 avg=677.976 std=159',
 'Running benchmark for at least 50 iterations and at least 1 seconds but terminate if exceeding 150 seconds.',
 'count=1404 first=646 curr=604 min=583 max=2027 avg=674.613 std=135',
 'Inference timings in us: Init: 154788, First inference: 2826, Warmup (avg): 677.976, Inference (avg): 674.613',
 'Note: as the benchmark tool itself affects memory footprint, the

In [12]:
def get_profiling_lines(lines):
    #print(lines)
    # find section split
    for i, line in enumerate(lines):
        #print(f"{i} : {line}")
    #if line.str.contains('= Run Order ='):
        if "Operator-wise Profiling " in line:
            split_line = i
            #print(f"Splitting text at line {split_line}")
            model_profiling = lines[:split_line]
            operator_profiling = lines[split_line+1:]

        else:
            pass
    if i==0:     
        print("WARNING: Could not find the text: 'Operator-wise Profiling Info for Regular Benchmark Runs:'")
        return (None, None)
    else:
        return model_profiling, operator_profiling


In [13]:
model_profiling, operator_profiling = get_profiling_lines(lines)

In [14]:
def get_index_of_model_profiling_lines(lines):
    model_lines_dict = {}
    for i, line in enumerate(lines):
        if "= Run Order =" in line:
            model_lines_dict["run_order"] = i
        elif "= Top by Computation Time =" in line:
            model_lines_dict["top_by_computation_time"] = i
        elif "The input model file size" in line:
            model_lines_dict["model_file_size"] = i
        elif "Initialized session in" in line:
            model_lines_dict["initialization_ms"] = i
        elif "Inference timings in us:" in line:
            model_lines_dict["inference_timings"] = i

    #print(model_lines_dict)
    return model_lines_dict


In [15]:
model_lines_dict = get_index_of_model_profiling_lines(model_profiling)

In [16]:
def get_model_profiling_stats(model_profiling):
    model_lines_dict = get_index_of_model_profiling_lines(model_profiling)

    model_stats = {}

    # extract model size
    model_size_string = model_profiling[model_lines_dict["model_file_size"]]
    model_size_string
    model_size= float(model_size_string.split(":")[-1].strip())
    model_stats["model_size_MB"] = model_size

    # extract inference times
    inference_string = model_profiling[model_lines_dict["inference_timings"]]
    #print(inference_string)
    inference_strings = inference_string.split(",")
    for item in inference_strings:
        item_time = item.split()[-1]
        #print(item_time)

    model_stats["init_us"] = int(inference_strings[0].split()[-1].strip())
    model_stats["first_inference_us"] = int(inference_strings[1].split()[-1].strip())
    model_stats["warmup_avg_us"] = float(inference_strings[2].split()[-1].strip())
    model_stats["inference_avg_us"] = float(inference_strings[3].split()[-1].strip())

    # model initialization timings
    init_string = model_profiling[model_lines_dict["initialization_ms"]]
    model_stats["initialization_ms"] = float(init_string.split()[-1].replace("ms.",""))


    # initialization run order
    init_run_order = model_profiling[model_lines_dict["run_order"]: model_lines_dict["top_by_computation_time"]]
    init_run_order = remove_tabs(init_run_order)

    model_stats["modify_graph_with_delegate_ms_first"] = float(init_run_order[2][1].strip())
    model_stats["modify_graph_with_delegate_ms_avg"] = float(init_run_order[2][2].strip())
    model_stats["modify_graph_with_delegate_ms_%"] = float(init_run_order[2][3].strip().replace("%", ""))
    model_stats["modify_graph_with_delegate_mem_KB"] = float(init_run_order[2][5].strip())

    model_stats["allocate_tensors_ms_first"] = float(init_run_order[3][1].strip())
    model_stats["allocate_tensors_ms_avg"] = float(init_run_order[3][2].strip())
    model_stats["allocate_tensors_ms_%"] = float(init_run_order[3][3].strip().replace("%", ""))

    return model_stats

In [17]:
model_stats = get_model_profiling_stats(model_profiling)
model_stats

{'model_size_MB': 0.07144,
 'init_us': 154788,
 'first_inference_us': 2826,
 'warmup_avg_us': 677.976,
 'inference_avg_us': 674.613,
 'initialization_ms': 154.788,
 'modify_graph_with_delegate_ms_first': 0.083,
 'modify_graph_with_delegate_ms_avg': 0.083,
 'modify_graph_with_delegate_ms_%': 64.341,
 'modify_graph_with_delegate_mem_KB': 0.0,
 'allocate_tensors_ms_first': 0.046,
 'allocate_tensors_ms_avg': 0.046,
 'allocate_tensors_ms_%': 35.659}

In [18]:
def get_profiling_stats_cpu(filepath):
    """
    Parse "bechmarking.txt" file that has been generated from a .tflite model with TensorFlow benchmarking tool.  

    The native benchmark binary for linux is available from this page:  
    [https://www.tensorflow.org/lite/performance/measurement](https://www.tensorflow.org/lite/performance/measurement)   

    Args:
        filepath (pathlib.Path): filepath to the extracted .txt file

    Returns:
        dict: dictionary with extracted model summary profiling information 
    """

    lines = clean_model_summary(filepath)
    model_profiling, operator_profiling = get_profiling_lines(lines)
    profiling_stats = get_model_profiling_stats(model_profiling)
    return profiling_stats


In [19]:
prof_stats = get_profiling_stats_cpu(models_benchmark_path)
prof_stats

{'model_size_MB': 0.07144,
 'init_us': 154788,
 'first_inference_us': 2826,
 'warmup_avg_us': 677.976,
 'inference_avg_us': 674.613,
 'initialization_ms': 154.788,
 'modify_graph_with_delegate_ms_first': 0.083,
 'modify_graph_with_delegate_ms_avg': 0.083,
 'modify_graph_with_delegate_ms_%': 64.341,
 'modify_graph_with_delegate_mem_KB': 0.0,
 'allocate_tensors_ms_first': 0.046,
 'allocate_tensors_ms_avg': 0.046,
 'allocate_tensors_ms_%': 35.659}

# Operator profiling

In [20]:
def get_operator_df(text, name=""):
    df = pd.DataFrame(text)
    df.rename(columns=df.iloc[0, :], inplace=True) 
    df.drop(df.index[0], inplace=True)
    df.columns = clean_column_names(df)
    # try:
    df["%"] = df["%"].apply(string_percent_to_float)
    # except:
    #     df["avg %"] = df["avg %"].apply(string_percent_to_float)
    # try:
    df["cdf%"] = df["cdf%"].apply(string_percent_to_float)
    # except:
    #     df["cdf %"] = df["cdf %"].apply(string_percent_to_float)
    df["first"] = df["first"].map(float)
    df["avg ms"] = df["avg ms"].map(float)
    df["mem KB"] = df["mem KB"].map(float)
    df["times called"] = df["times called"].map(int)#
    #df.Name = name

    return df

In [21]:
def get_node_df(text, name=""):
    df = pd.DataFrame(text)
    df.rename(columns=df.iloc[0, :], inplace=True) 
    df.drop(df.index[0], inplace=True)
    df.columns = clean_column_names(df)
    df["avg %"] = df["avg %"].apply(string_percent_to_float)
    df["cdf %"] = df["cdf %"].apply(string_percent_to_float)
    df["avg ms"] = df["avg ms"].map(float)
    df["mem KB"] = df["mem KB"].map(float)
    df["times called"] = df["times called"].map(int)
    df["count"] = df["count"].map(int)
    df.Name = name

    return df

In [22]:
def get_index_of_operator_profiling_lines(lines):
    operator_lines_dict = {}
    for i, line in enumerate(lines):
        #if line.str.contains('= Run Order ='):
        if "= Run Order =" in line:
            operator_lines_dict["run_order"] = i
        elif "= Top by Computation Time =" in line:
            operator_lines_dict["top_by_computation_time"] = i
        elif "= Summary by node type =" in line:
            operator_lines_dict["summary_by_node_type"] = i

    # print(operator_lines_dict)
    return operator_lines_dict

In [23]:
# operator_lines_dict = get_index_of_operator_profiling_lines(operator_profiling)
# operator_lines_dict

In [24]:
def get_summary_by_node_type_df(operator_profiling, operator_lines_dict, model_name):
    summary_by_node_type = operator_profiling[operator_lines_dict["summary_by_node_type"] +1:-3]
    summary_by_node_type = remove_tabs(summary_by_node_type)
    summary_by_node_type_df = get_node_df(summary_by_node_type, name=f"Summary by node type - {model_name}")
    return summary_by_node_type_df

In [25]:
def get_operator_top_by_comp_time_df(operator_profiling, operator_lines_dict, model_name):

    top_by_comp_time = operator_profiling[
        operator_lines_dict["top_by_computation_time"]+ 1 : operator_lines_dict["summary_by_node_type"]- 1]
    top_by_comp_time_clean = remove_tabs(top_by_comp_time)

    df = pd.DataFrame(top_by_comp_time_clean)

    df.rename(columns=df.iloc[0, :], inplace=True)
    df.drop(df.index[0], inplace=True)
    df.columns = clean_column_names(df)

    df["%"] = df["%"].apply(string_percent_to_float)
    df["cdf%"] = df["cdf%"].apply(string_percent_to_float)
    df["first"] = df["first"].map(float)
    df["avg ms"] = df["avg ms"].map(float)
    df["mem KB"] = df["mem KB"].map(float)
    df["times called"] = df["times called"].map(int)

    return df

In [26]:
def get_run_order_df(operator_lines_dict, operator_profiling, model_name):
    run_order = operator_profiling[operator_lines_dict["run_order"] +1: operator_lines_dict["top_by_computation_time"]]
    run_order = remove_tabs(run_order)
    df = get_operator_df(run_order, name= f"Run order - {model_name}")
    return df

In [27]:
def get_profiling_dataframes_cpu(filepath, model_name):
    """
    Parse "bechmarking.txt" file that has been generated from a .tflite model with TensorFlow benchmarking tool.  

    The native benchmark binary for linux is available from this page:  
    [https://www.tensorflow.org/lite/performance/measurement](https://www.tensorflow.org/lite/performance/measurement)   

    Args:
        filepath (pathlib.Path): filepath to the extracted .txt file

    Returns:
        list(pd.DataFrame, pd.DataFrame, pd.DataFrame): tuple containing the following dataframes:

            operator_run_order_df, operator_by_comp_time_df, summary_node_type_df
    """

    lines = clean_model_summary(filepath)
    model_profiling, operator_profiling = get_profiling_lines(lines)
    operator_lines_dict = get_index_of_operator_profiling_lines(operator_profiling)

    operator_run_order_df = get_run_order_df(operator_lines_dict, operator_profiling, model_name)
    operator_by_comp_time_df = get_operator_top_by_comp_time_df(operator_profiling, operator_lines_dict, model_name)
    summary_node_type_df = get_summary_by_node_type_df(operator_profiling, operator_lines_dict, model_name)

    return operator_run_order_df, operator_by_comp_time_df, summary_node_type_df

In [28]:
operator_run_order_df, operator_by_comp_time_df, summary_node_type_df = get_profiling_dataframes_cpu(models_benchmark_path, model_name)

In [29]:
operator_run_order_df

,node type,first,avg ms,%,cdf%,mem KB,times called,Name
1,QUANTIZE,0.104,0.105,15.945,15.945,0.0,1,[tfl.quantize]:0
2,CONV_2D,0.094,0.091,13.832,29.777,0.0,1,[mobilenetv1/re_lu/Relu;mobilenetv1/batch_normalization/FusedBatchNormV3;mobilenetv1/conv2d/BiasAdd/ReadVariableOp;mobilenetv1/conv2d/BiasAdd;mobilenetv1/conv2d_11/Conv2D;mobilenetv1/conv2d/Conv2D1]:1
3,DEPTHWISE_CONV_2D,0.108,0.115,17.609,47.386,0.0,1,[mobilenetv1/re_lu_1/Relu;mobilenetv1/batch_normalization_1/FusedBatchNormV3;mobilenetv1/depthwise_conv2d/depthwise;mobilenetv1/depthwise_conv2d/BiasAdd/ReadVariableOp;mobilenetv1/depthwise_conv2d/BiasAdd;mobilenetv1/conv2d_11/Conv2D1]:2
4,CONV_2D,0.043,0.045,6.925,54.312,0.0,1,[mobilenetv1/re_lu_2/Relu;mobilenetv1/batch_normalization_2/FusedBatchNormV3;mobilenetv1/conv2d_1/BiasAdd/ReadVariableOp;mobilenetv1/conv2d_1/BiasAdd;mobilenetv1/depthwise_conv2d_1/depthwise;mobilenetv1/conv2d_1/Conv2D1]:3
5,DEPTHWISE_CONV_2D,0.046,0.050,7.618,61.929,0.0,1,[mobilenetv1/re_lu_3/Relu;mobilenetv1/batch_normalization_3/FusedBatchNormV3;mobilenetv1/depthwise_conv2d_1/depthwise;mobilenetv1/depthwise_conv2d_1/BiasAdd/ReadVariableOp;mobilenetv1/depthwise_conv2d_1/BiasAdd1]:4
6,CONV_2D,0.015,0.017,2.559,64.488,0.0,1,[mobilenetv1/re_lu_4/Relu;mobilenetv1/batch_normalization_4/FusedBatchNormV3;mobilenetv1/conv2d_2/BiasAdd/ReadVariableOp;mobilenetv1/conv2d_2/BiasAdd;mobilenetv1/depthwise_conv2d_3/depthwise;mobilenetv1/conv2d_2/Conv2D1]:5
7,DEPTHWISE_CONV_2D,0.046,0.050,7.589,72.077,0.0,1,[mobilenetv1/re_lu_5/Relu;mobilenetv1/batch_normalization_5/FusedBatchNormV3;mobilenetv1/depthwise_conv2d_2/depthwise;mobilenetv1/depthwise_conv2d_2/BiasAdd/ReadVariableOp;mobilenetv1/depthwise_conv2d_2/BiasAdd;mobilenetv1/depthwise_conv2d_3/depthwise1]:6
8,CONV_2D,0.017,0.018,2.728,74.805,0.0,1,[mobilenetv1/re_lu_6/Relu;mobilenetv1/batch_normalization_6/FusedBatchNormV3;mobilenetv1/conv2d_3/BiasAdd/ReadVariableOp;mobilenetv1/conv2d_3/BiasAdd;mobilenetv1/depthwise_conv2d_3/depthwise;mobilenetv1/conv2d_3/Conv2D1]:7
9,DEPTHWISE_CONV_2D,0.019,0.020,3.001,77.807,0.0,1,[mobilenetv1/re_lu_7/Relu;mobilenetv1/batch_normalization_7/FusedBatchNormV3;mobilenetv1/depthwise_conv2d_3/depthwise;mobilenetv1/depthwise_conv2d_3/BiasAdd/ReadVariableOp;mobilenetv1/depthwise_conv2d_3/BiasAdd1]:8
10,CONV_2D,0.007,0.008,1.189,78.996,0.0,1,[mobilenetv1/re_lu_8/Relu;mobilenetv1/batch_normalization_8/FusedBatchNormV3;mobilenetv1/conv2d_4/BiasAdd/ReadVariableOp;mobilenetv1/conv2d_4/BiasAdd;mobilenetv1/depthwise_conv2d_5/depthwise;mobilenetv1/conv2d_4/Conv2D1]:9


In [30]:
operator_by_comp_time_df

,node type,first,avg ms,%,cdf%,mem KB,times called,Name
1,DEPTHWISE_CONV_2D,0.108,0.115,17.609,17.609,0.0,1,[mobilenetv1/re_lu_1/Relu;mobilenetv1/batch_normalization_1/FusedBatchNormV3;mobilenetv1/depthwise_conv2d/depthwise;mobilenetv1/depthwise_conv2d/BiasAdd/ReadVariableOp;mobilenetv1/depthwise_conv2d/BiasAdd;mobilenetv1/conv2d_11/Conv2D1]:2
2,QUANTIZE,0.104,0.105,15.945,33.554,0.0,1,[tfl.quantize]:0
3,CONV_2D,0.094,0.091,13.832,47.386,0.0,1,[mobilenetv1/re_lu/Relu;mobilenetv1/batch_normalization/FusedBatchNormV3;mobilenetv1/conv2d/BiasAdd/ReadVariableOp;mobilenetv1/conv2d/BiasAdd;mobilenetv1/conv2d_11/Conv2D;mobilenetv1/conv2d/Conv2D1]:1
4,DEPTHWISE_CONV_2D,0.046,0.050,7.618,55.004,0.0,1,[mobilenetv1/re_lu_3/Relu;mobilenetv1/batch_normalization_3/FusedBatchNormV3;mobilenetv1/depthwise_conv2d_1/depthwise;mobilenetv1/depthwise_conv2d_1/BiasAdd/ReadVariableOp;mobilenetv1/depthwise_conv2d_1/BiasAdd1]:4
5,DEPTHWISE_CONV_2D,0.046,0.050,7.589,62.593,0.0,1,[mobilenetv1/re_lu_5/Relu;mobilenetv1/batch_normalization_5/FusedBatchNormV3;mobilenetv1/depthwise_conv2d_2/depthwise;mobilenetv1/depthwise_conv2d_2/BiasAdd/ReadVariableOp;mobilenetv1/depthwise_conv2d_2/BiasAdd;mobilenetv1/depthwise_conv2d_3/depthwise1]:6
6,CONV_2D,0.043,0.045,6.925,69.518,0.0,1,[mobilenetv1/re_lu_2/Relu;mobilenetv1/batch_normalization_2/FusedBatchNormV3;mobilenetv1/conv2d_1/BiasAdd/ReadVariableOp;mobilenetv1/conv2d_1/BiasAdd;mobilenetv1/depthwise_conv2d_1/depthwise;mobilenetv1/conv2d_1/Conv2D1]:3
7,DEPTHWISE_CONV_2D,0.028,0.030,4.590,74.108,0.0,1,[mobilenetv1/re_lu_9/Relu;mobilenetv1/batch_normalization_9/FusedBatchNormV3;mobilenetv1/depthwise_conv2d_4/depthwise;mobilenetv1/depthwise_conv2d_4/BiasAdd/ReadVariableOp;mobilenetv1/depthwise_conv2d_4/BiasAdd;mobilenetv1/depthwise_conv2d_5/depthwise1]:10
8,DEPTHWISE_CONV_2D,0.019,0.020,3.001,77.109,0.0,1,[mobilenetv1/re_lu_7/Relu;mobilenetv1/batch_normalization_7/FusedBatchNormV3;mobilenetv1/depthwise_conv2d_3/depthwise;mobilenetv1/depthwise_conv2d_3/BiasAdd/ReadVariableOp;mobilenetv1/depthwise_conv2d_3/BiasAdd1]:8
9,CONV_2D,0.017,0.018,2.728,79.838,0.0,1,[mobilenetv1/re_lu_6/Relu;mobilenetv1/batch_normalization_6/FusedBatchNormV3;mobilenetv1/conv2d_3/BiasAdd/ReadVariableOp;mobilenetv1/conv2d_3/BiasAdd;mobilenetv1/depthwise_conv2d_3/depthwise;mobilenetv1/conv2d_3/Conv2D1]:7
10,CONV_2D,0.015,0.017,2.559,82.396,0.0,1,[mobilenetv1/re_lu_4/Relu;mobilenetv1/batch_normalization_4/FusedBatchNormV3;mobilenetv1/conv2d_2/BiasAdd/ReadVariableOp;mobilenetv1/conv2d_2/BiasAdd;mobilenetv1/depthwise_conv2d_3/depthwise;mobilenetv1/conv2d_2/Conv2D1]:5


In [31]:
summary_node_type_df

,Node type,count,avg ms,avg %,cdf %,mem KB,times called
1,DEPTHWISE_CONV_2D,10,0.309,48.433,48.433,0.0,10
2,CONV_2D,12,0.221,34.639,83.072,0.0,12
3,QUANTIZE,2,0.104,16.301,99.373,0.0,2
4,MEAN,1,0.002,0.313,99.687,0.0,1
5,STRIDED_SLICE,1,0.001,0.157,99.843,0.0,1
6,PACK,1,0.001,0.157,100.000,0.0,1
7,SOFTMAX,1,0.000,0.000,100.000,0.0,1
8,SHAPE,1,0.000,0.000,100.000,0.0,1
9,RESHAPE,1,0.000,0.000,100.000,0.0,1


In [32]:
# operator_profiling = lines[split_line+1:]
# operator_profiling

In [33]:
# operator_lines_dict = {}
# for i, line in enumerate(operator_profiling):
#     #if line.str.contains('= Run Order ='):
#     if "= Run Order =" in line:
#         operator_lines_dict["run_order"] = i
#     elif "= Top by Computation Time =" in line:
#         operator_lines_dict["top_by_computation_time"] = i
#     elif "= Summary by node type =" in line:
#         operator_lines_dict["summary_by_node_type"] = i

# operator_lines_dict

In [34]:
# summary_by_node_type = operator_profiling[operator_lines_dict["summary_by_node_type"] +1:-3]
# summary_by_node_type = remove_tabs(summary_by_node_type)
# summary_by_node_type

In [35]:
# summary_by_node_type_df = get_node_df(summary_by_node_type, name=f"Summary by node type - {model_name}")
# summary_by_node_type_df

In [36]:
#summary_by_node_type_df.Name

In [37]:
#summary_by_node_type_df.info

In [38]:
# top_by_comp_time = operator_profiling[operator_lines_dict["top_by_computation_time"] +1: operator_lines_dict["summary_by_node_type"]-1]
# top_by_comp_time = remove_tabs(top_by_comp_time)
# top_by_comp_time

In [39]:
# df_operator_top_by_comp_time = get_operator_df(top_by_comp_time, name=f"Top by compute time - {model_name}")
# df_operator_top_by_comp_time

In [40]:
# run_order = operator_profiling[operator_lines_dict["run_order"] +1: operator_lines_dict["top_by_computation_time"]]
# run_order = remove_tabs(run_order)
# #run_order

In [41]:
# df_operator_run_order = get_operator_df(run_order, name= f"Run order - {model_name}")
# #df_operator_run_order.style.set_properties(**{'text-align': 'left'})
# df_operator_run_order.head(20)

In [42]:
# df_operator_run_order.info()

# Logging to wandb

In [47]:
# Generate run ids



# PROJECT = model_name.split("_")[0]

PROJECT = "model_DB"

run = wandb.init(
        # Set the project where this run will be logged
        project=PROJECT, 
        name = model_name,
        id = id, 
        resume="allow",
        sync_tensorboard=True
        )
# Specify the configuration variables
config = wandb.config
config.architecture = model_name

run.log(model_stats)
run.log(prof_stats)

# # Create a table
operator_top_by_time_table = wandb.Table(dataframe=operator_by_comp_time_df)
# operator_top_by_time_artifact = wandb.Artifact(name=f'{model_name}_operator_top_by_time', type='dataframe')
# operator_top_by_time_artifact.add(operator_top_by_time_table, "Profiling operators - top by time")
# run.log_artifact(operator_top_by_time_artifact)

summary_by_node_type_table = wandb.Table(dataframe=summary_node_type_df)
# summary_by_node_type_artifact = wandb.Artifact(name=f'{model_name}_summary_by_node_type', type='dataframe')
# summary_by_node_type_artifact.add(summary_by_node_type_table, "Node type summary")
# run.log_artifact(summary_by_node_type_artifact)

operator_run_order_table = wandb.Table(dataframe=operator_run_order_df)
# operator_run_order_artifact = wandb.Artifact(name=f'{model_name}_operator_run_order', type='dataframe')
# operator_run_order_artifact.add(operator_run_order_table, "Run order")
# run.log_artifact(operator_run_order_artifact)


run.log({"Profiling operators - top by time": operator_top_by_time_table})
run.log({"Node type summary" : summary_by_node_type_table})
run.log({"Run order" : operator_run_order_table})
# run.log({"Chart_table": table})

# table = wandb.Table(columns = ["plotly_figure"])

# # Create path for Plotly figure
# path_to_plotly_html = "./plotly_figure.html"

# # Write Plotly figure to HTML
# fig.write_html(path_to_plotly_html, auto_play = False) # Setting auto_play to False prevents animated Plotly charts from playing in the table automatically

# # Add Plotly figure as HTML file into Table
# table.add_data(wandb.Html(path_to_plotly_html))

# # Log Table
# run.log({"Chart_table": table})

wandb.log(model_stats)

# #wandb.log({"Peak activations chart": fig})

wandb.finish()

allocate_tensors_ms_%,▁▁▁
allocate_tensors_ms_avg,▁▁▁
allocate_tensors_ms_first,▁▁▁
first_inference_us,▁▁▁
inference_avg_us,▁▁▁
init_us,▁▁▁
initialization_ms,▁▁▁
model_size_MB,▁▁▁
modify_graph_with_delegate_mem_KB,▁▁▁
modify_graph_with_delegate_ms_%,▁▁▁
modify_graph_with_delegate_ms_avg,▁▁▁
